In [32]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 25)

In [2]:
df = pd.read_csv("SalaryData.csv")
TeamsWins = pd.read_csv("TeamsWins.csv")
Teams = ["arizona-cardinals", "atlanta-falcons", 'baltimore-ravens', "buffalo-bills", "carolina-panthers", "chicago-bears",
        "cincinnati-bengals", "cleveland-browns", "dallas-cowboys", "denver-broncos", "detroit-lions", "green-bay-packers",
        "houston-texans", "indianapolis-colts", "jacksonville-jaguars", "kansas-city-chiefs", "oakland-raiders", "san-diego-chargers",
        "los-angeles-rams", "miami-dolphins", "minnesota-vikings", "new-england-patriots", "new-orleans-saints", "new-york-giants",
        "new-york-jets", "philadelphia-eagles", "pittsburgh-steelers", "san-francisco-49ers", "seattle-seahawks", "tampa-bay-buccaneers",
        "tennessee-titans", "washington-redskins"]
Years = range(2005,2020)

## Large Scale Changes
#### 1. 2010 Adjustment
#### 2. Removal of "-"
#### 3. Take Dollar Sign out of Cash

In [3]:
df.dtypes

CapPerc      float64
Cash          object
DeadCap        int64
IR             int64
Player        object
Position      object
Prac           int64
SalaryCap     object
Susp           int64
Team          object
Year           int64
dtype: object

In [4]:
##Little Cleaning
df.replace("-", value = 0, inplace = True)
df.replace("- ", value = 0, inplace = True)
def DropDol(x):
    CashPre = str(x['Cash'])
    Cash = CashPre.lstrip('$').replace(',','')
    RetCash = int(Cash)
    return RetCash

df['Cash'] = df.apply(DropDol, axis = 1)

# Create Final DataFrames (Sorted by Team, Year, Position Groups)
### 3 buckets (Offense, Defense, Special Teams)

In [5]:
#### Because there was no cap in 2010, I decided to use the average cap of 2008-2012
def Cap2010Solve(x):
    Cap2010 = (116000000 + 123000000 + 120000000 + 120600000)/4
    if x['Year'] == 2010:
        CapPerc = float(x['SalaryCap'])/Cap2010*100
        
    else:
        CapPerc = x['CapPerc']
    return CapPerc


df['CapPerc'] = df.apply(Cap2010Solve, axis = 1)

In [6]:
df['Position'].unique()

array(['DE', 'TE', 'CB', 'G', 'K', 'LB', 'QB', 'T', 'RB', 'S', 'C', 'WR',
       'LS', 'DT', 'OLB', 'SS', 'LT', 'ILB', 'RT', 'FS', 'FB', 'P', 'KR',
       'OL', 'PR'], dtype=object)

In [7]:
##Most simple position groups. Offense, Defense, SpecTeams

df_3buckets = df.copy()

def Pos_3buck(x):
    if x['Position'] in ['DE','CB','LB','S','DT','OLB','SS','ILB','FS']:
        Group = 'DEF'
    elif x['Position'] in ['TE','G','QB','T','RB','WR','LT','RT','FB','C','OL']:
        Group = 'OFF'
    elif x['Position'] in ['K','P','KR','PR','LS']:
        Group = 'Special'
    else:
        Group = "Error"
        
    return Group


df_3buckets['Group'] = df_3buckets.apply(Pos_3buck,axis = 1)

In [8]:
#Allow for groupby
convert_dict = {'Cash': float, 'Player': str,'Position': str, "SalaryCap": float,"Team": str,'Group': str}
df_3buckets = df_3buckets.astype(convert_dict)

In [9]:
df_3buckets.head()

,CapPerc,Cash,DeadCap,IR,Player,Position,Prac,SalaryCap,Susp,Team,Year,Group
0,0.199333,0.0,1,0,Cody Brown,DE,0,239000.0,0,arizona-cardinals,2010,DEF
1,0.041701,0.0,1,0,Anthony Becht,TE,0,50000.0,0,arizona-cardinals,2010,OFF
2,0.018397,0.0,1,0,Jorrick Calvin,CB,0,22058.0,0,arizona-cardinals,2010,DEF
3,0.014721,0.0,1,0,Trevor Canfield,G,0,17650.0,0,arizona-cardinals,2010,OFF
4,1.042535,0.0,1,0,Jason Elam,K,0,1250000.0,0,atlanta-falcons,2010,Special


In [10]:
GB3Buck = pd.DataFrame(columns = ['Year','Team','Off','Def',"Special",'Dead','IR','Prac','Susp'])
for Team in Teams:
    for Year in Years:
        Off = round(df_3buckets[(df_3buckets['Team'] == Team)&(df_3buckets['Year'] == Year)&(df_3buckets['Group'] == 'OFF')]['CapPerc'].sum(),2)
        Def = round(df_3buckets[(df_3buckets['Team'] == Team)&(df_3buckets['Year'] == Year)&(df_3buckets['Group'] == 'DEF')]['CapPerc'].sum(),2)
        Special = round(df_3buckets[(df_3buckets['Team'] == Team)&(df_3buckets['Year'] == Year)&(df_3buckets['Group'] == 'Special')]['CapPerc'].sum(),2)
        Dead = round(df_3buckets[(df_3buckets['Team'] == Team)&(df_3buckets['Year'] == Year)&(df_3buckets['DeadCap'] == 1)]['CapPerc'].sum(),2)
        IR = round(df_3buckets[(df_3buckets['Team'] == Team)&(df_3buckets['Year'] == Year)&(df_3buckets['IR'] == 1)]['CapPerc'].sum(),2)
        Prac = round(df_3buckets[(df_3buckets['Team'] == Team)&(df_3buckets['Year'] == Year)&(df_3buckets['Prac'] == 1)]['CapPerc'].sum(),2)
        Susp = round(df_3buckets[(df_3buckets['Team'] == Team)&(df_3buckets['Year'] == Year)&(df_3buckets['Susp'] == 1)]['CapPerc'].sum(),2)
        NewRow = {'Year': Year, 'Team': Team, 'Off': Off, 'Def': Def, 'Special': Special, 'Dead': Dead, 'IR': IR, 'Prac': Prac, 'Susp': Susp}
        GB3Buck = GB3Buck.append(NewRow, ignore_index = True)

In [11]:
## Change team and mascot to just mascot, which never changes
def JustMascot(x):
    Team = x['Team']
    Mascot = Team.split('-')[-1]
    return Mascot
    
    
    
GB3Buck['Team'] = GB3Buck.apply(JustMascot, axis = 1)

cardinals
cardinals
cardinals
cardinals
cardinals
cardinals
cardinals
cardinals
cardinals
cardinals
cardinals
cardinals
cardinals
cardinals
cardinals
falcons
falcons
falcons
falcons
falcons
falcons
falcons
falcons
falcons
falcons
falcons
falcons
falcons
falcons
falcons
ravens
ravens
ravens
ravens
ravens
ravens
ravens
ravens
ravens
ravens
ravens
ravens
ravens
ravens
ravens
bills
bills
bills
bills
bills
bills
bills
bills
bills
bills
bills
bills
bills
bills
bills
panthers
panthers
panthers
panthers
panthers
panthers
panthers
panthers
panthers
panthers
panthers
panthers
panthers
panthers
panthers
bears
bears
bears
bears
bears
bears
bears
bears
bears
bears
bears
bears
bears
bears
bears
bengals
bengals
bengals
bengals
bengals
bengals
bengals
bengals
bengals
bengals
bengals
bengals
bengals
bengals
bengals
browns
browns
browns
browns
browns
browns
browns
browns
browns
browns
browns
browns
browns
browns
browns
cowboys
cowboys
cowboys
cowboys
cowboys
cowboys
cowboys
cowboys
cowboys
cowboys
cowbo

In [12]:
## Merge team salary info with the success
Df3Buck = GB3Buck.merge(TeamsWins, how = "inner", on = ['Year','Team'])
Df3Buck.to_csv("3Buckets.csv", index = False)

### 5 Buckets (Oline, Fantasy, Front7, Dbacks)

In [16]:
##Slightly more advanced position groups

df_5buckets = df.copy()

def Pos_5buck(x):
    if x['Position'] in ['CB','S','SS','FS']:
        Group = 'Dbacks'
    elif x['Position'] in ['DE','LB','DT','OLB','ILB']:
        Group = 'Front7'
    elif x['Position'] in ['TE','QB','RB','WR','FB']:
        Group = 'Fantasy'
    elif x['Position'] in ['G','T','LT','RT','C','OL']:
        Group = 'Oline'
    elif x['Position'] in ['K','P','KR','PR','LS']:
        Group = 'Special'
    else:
        Group = "Error"
        
    return Group


df_5buckets['Group'] = df_5buckets.apply(Pos_5buck,axis = 1)

In [17]:
#Allow for groupby
convert_dict = {'Cash': float, 'Player': str,'Position': str, "SalaryCap": float,"Team": str,'Group': str}
df_5buckets = df_5buckets.astype(convert_dict)

In [18]:
GB5Buck = pd.DataFrame(columns = ['Year','Team','Dbacks','Front7','Fantasy','Oline',"Special",'Dead','IR','Prac','Susp'])
for Team in Teams:
    for Year in Years:
        Dbacks = round(df_5buckets[(df_5buckets['Team'] == Team)&(df_5buckets['Year'] == Year)&(df_5buckets['Group'] == 'Dbacks')]['CapPerc'].sum(),2)
        Front7 = round(df_5buckets[(df_5buckets['Team'] == Team)&(df_5buckets['Year'] == Year)&(df_5buckets['Group'] == 'Front7')]['CapPerc'].sum(),2)
        Fantasy = round(df_5buckets[(df_5buckets['Team'] == Team)&(df_5buckets['Year'] == Year)&(df_5buckets['Group'] == 'Fantasy')]['CapPerc'].sum(),2)
        Oline = round(df_5buckets[(df_5buckets['Team'] == Team)&(df_5buckets['Year'] == Year)&(df_5buckets['Group'] == 'Oline')]['CapPerc'].sum(),2)
        Special = round(df_5buckets[(df_5buckets['Team'] == Team)&(df_5buckets['Year'] == Year)&(df_5buckets['Group'] == 'Special')]['CapPerc'].sum(),2)
        Dead = round(df_5buckets[(df_5buckets['Team'] == Team)&(df_5buckets['Year'] == Year)&(df_5buckets['DeadCap'] == 1)]['CapPerc'].sum(),2)
        IR = round(df_5buckets[(df_5buckets['Team'] == Team)&(df_5buckets['Year'] == Year)&(df_5buckets['IR'] == 1)]['CapPerc'].sum(),2)
        Prac = round(df_5buckets[(df_5buckets['Team'] == Team)&(df_5buckets['Year'] == Year)&(df_5buckets['Prac'] == 1)]['CapPerc'].sum(),2)
        Susp = round(df_5buckets[(df_5buckets['Team'] == Team)&(df_5buckets['Year'] == Year)&(df_5buckets['Susp'] == 1)]['CapPerc'].sum(),2)
        NewRow = {'Year': Year, 'Team': Team, 'Dbacks': Dbacks, 'Front7': Front7, 'Fantasy': Fantasy, 'Oline': Oline, 'Special': Special, 'Dead': Dead, 'IR': IR, 'Prac': Prac, 'Susp': Susp}
        GB5Buck = GB5Buck.append(NewRow, ignore_index = True)

In [19]:
## Change team and mascot to just mascot, which never changes
def JustMascot(x):
    Team = x['Team']
    Mascot = Team.split('-')[-1]
    return Mascot
    
    
    
GB5Buck['Team'] = GB5Buck.apply(JustMascot, axis = 1)

In [20]:
## Merge team salary info with the success
GB5Buck = GB5Buck.merge(TeamsWins, how = "inner", on = ['Year','Team'])
GB5Buck.to_csv("5Buckets.csv", index = False)

### 8 Buckets (Dline, Linebackers, Dbacks, Oline, PassCatchers,QB,RB)

In [21]:
##Slightly more advanced position groups

df_8buckets = df.copy()

def Pos_8buck(x):
    if x['Position'] in ['CB','S','SS','FS']:
        Group = 'Dbacks'
    elif x['Position'] in ['LB','OLB','ILB']:
        Group = 'LineBackers'
    elif x['Position'] in ['DE','DT']:
        Group = 'Dline'
    elif x['Position'] in ['TE','WR']:
        Group = 'PassCatchers'
    elif x['Position'] in ['G','T','LT','RT','C','OL']:
        Group = 'Oline'
    elif x['Position'] in ['QB']:
        Group = 'QB'
    elif x['Position'] in ['RB']:
        Group = 'RB'
    elif x['Position'] in ['K','P','KR','PR','LS']:
        Group = 'Special'
    else:
        Group = "Error"
        
    return Group


df_8buckets['Group'] = df_8buckets.apply(Pos_8buck,axis = 1)

In [22]:
#Allow for groupby
convert_dict = {'Cash': float, 'Player': str,'Position': str, "SalaryCap": float,"Team": str,'Group': str}
df_8buckets = df_8buckets.astype(convert_dict)

In [23]:
GB8Buck = pd.DataFrame(columns = ['Year','Team','Dbacks','LineBackers','Dline','PassCatchers','Oline','QB','RB',"Special",'Dead','IR','Prac','Susp'])
for Team in Teams:
    for Year in Years:
        Dbacks = round(df_8buckets[(df_8buckets['Team'] == Team)&(df_8buckets['Year'] == Year)&(df_8buckets['Group'] == 'Dbacks')]['CapPerc'].sum(),2)
        LineBackers = round(df_8buckets[(df_8buckets['Team'] == Team)&(df_8buckets['Year'] == Year)&(df_8buckets['Group'] == 'LineBackers')]['CapPerc'].sum(),2)
        Dline = round(df_8buckets[(df_8buckets['Team'] == Team)&(df_8buckets['Year'] == Year)&(df_8buckets['Group'] == 'Dline')]['CapPerc'].sum(),2)
        Oline = round(df_8buckets[(df_8buckets['Team'] == Team)&(df_8buckets['Year'] == Year)&(df_8buckets['Group'] == 'Oline')]['CapPerc'].sum(),2)
        PassCatchers = round(df_8buckets[(df_8buckets['Team'] == Team)&(df_8buckets['Year'] == Year)&(df_8buckets['Group'] == 'PassCatchers')]['CapPerc'].sum(),2)
        QB = round(df_8buckets[(df_8buckets['Team'] == Team)&(df_8buckets['Year'] == Year)&(df_8buckets['Group'] == 'QB')]['CapPerc'].sum(),2)
        RB = round(df_8buckets[(df_8buckets['Team'] == Team)&(df_8buckets['Year'] == Year)&(df_8buckets['Group'] == 'RB')]['CapPerc'].sum(),2)
        Special = round(df_8buckets[(df_8buckets['Team'] == Team)&(df_8buckets['Year'] == Year)&(df_8buckets['Group'] == 'Special')]['CapPerc'].sum(),2)
        Dead = round(df_8buckets[(df_8buckets['Team'] == Team)&(df_8buckets['Year'] == Year)&(df_8buckets['DeadCap'] == 1)]['CapPerc'].sum(),2)
        IR = round(df_8buckets[(df_8buckets['Team'] == Team)&(df_8buckets['Year'] == Year)&(df_8buckets['IR'] == 1)]['CapPerc'].sum(),2)
        Prac = round(df_8buckets[(df_8buckets['Team'] == Team)&(df_8buckets['Year'] == Year)&(df_8buckets['Prac'] == 1)]['CapPerc'].sum(),2)
        Susp = round(df_8buckets[(df_8buckets['Team'] == Team)&(df_8buckets['Year'] == Year)&(df_8buckets['Susp'] == 1)]['CapPerc'].sum(),2)
        NewRow = {'Year': Year, 'Team': Team, 'Dbacks': Dbacks,'LineBackers': LineBackers,'Dline': Dline,'PassCatchers': PassCatchers,'Oline': Oline,'QB': QB,'RB': RB, 'Special': Special, 'Dead': Dead, 'IR': IR, 'Prac': Prac, 'Susp': Susp}
        GB8Buck = GB8Buck.append(NewRow, ignore_index = True)

In [24]:
## Change team and mascot to just mascot, which never changes
def JustMascot(x):
    Team = x['Team']
    Mascot = Team.split('-')[-1]
    return Mascot
    
    
    
GB8Buck['Team'] = GB8Buck.apply(JustMascot, axis = 1)

In [25]:
## Merge team salary info with the success
GB8Buck = GB8Buck.merge(TeamsWins, how = "inner", on = ['Year','Team'])
GB8Buck.to_csv("8Buckets.csv", index = False)

### 12 Buckets (Safety, Corner, EDGE (OLB + DE), ILB, DT, WR, TE, T, InteriorLine (C,G,OL), QB, RB, Special Teams)

In [3]:
##Slightly more advanced position groups

df_12buckets = df.copy()

def Pos_12buck(x):
    if x['Position'] in ['S','SS','FS']:
        Group = 'Safeties'
    elif x['Position'] in ['CB']:
        Group = 'Corners'
    elif x['Position'] in ['LB','ILB']:
        Group = 'InsideBackers'
    elif x['Position'] in ['DT']:
        Group = 'InteriorDLine'
    elif x['Position'] in ['OLB','DE']:
        Group = 'Edge'
    elif x['Position'] in ['TE']:
        Group = 'TEs'
    elif x['Position'] in ['WR']:
        Group = 'WRs'
    elif x['Position'] in ['G','C','OL']:
        Group = 'InteriorOLine'
    elif x['Position'] in ['T','LT','RT']:
        Group = 'Tackles'
    elif x['Position'] in ['QB']:
        Group = 'QB'
    elif x['Position'] in ['RB']:
        Group = 'RB'
    elif x['Position'] in ['K','P','KR','PR','LS']:
        Group = 'Special'
    else:
        Group = "Error"
        
    return Group


df_12buckets['Group'] = df_12buckets.apply(Pos_12buck,axis = 1)

In [4]:
#Allow for groupby
convert_dict = {'Cash': float, 'Player': str,'Position': str, "SalaryCap": float,"Team": str,'Group': str}
df_12buckets = df_12buckets.astype(convert_dict)

ValueError: could not convert string to float: '-'

In [ ]:
GB12Buck = pd.DataFrame(columns = ['Year','Team','Safeties','Corners','InsideBackers','Edge','InteriorDLine','WRs','TEs','InteriorOLine','Tackles','QB','RB',"Special",'Dead','IR','Prac','Susp'])
for Team in Teams:
    for Year in Years:
        Safeties = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['Group'] == 'Safeties')]['CapPerc'].sum(),2)
        Corners = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['Group'] == 'Corners')]['CapPerc'].sum(),2)
        InsideBackers = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['Group'] == 'InsideBackers')]['CapPerc'].sum(),2)
        Edge = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['Group'] == 'Edge')]['CapPerc'].sum(),2)
        InteriorDLine = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['Group'] == 'InteriorDLine')]['CapPerc'].sum(),2)
        InteriorOLine = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['Group'] == 'InteriorOLine')]['CapPerc'].sum(),2)
        Tackles = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['Group'] == 'Tackles')]['CapPerc'].sum(),2)
        WRs = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['Group'] == 'WRs')]['CapPerc'].sum(),2)
        TEs = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['Group'] == 'TEs')]['CapPerc'].sum(),2)
        QB = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['Group'] == 'QB')]['CapPerc'].sum(),2)
        RB = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['Group'] == 'RB')]['CapPerc'].sum(),2)
        Special = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['Group'] == 'Special')]['CapPerc'].sum(),2)
        Dead = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['DeadCap'] == 1)]['CapPerc'].sum(),2)
        IR = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['IR'] == 1)]['CapPerc'].sum(),2)
        Prac = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['Prac'] == 1)]['CapPerc'].sum(),2)
        Susp = round(df_12buckets[(df_12buckets['Team'] == Team)&(df_12buckets['Year'] == Year)&(df_12buckets['Susp'] == 1)]['CapPerc'].sum(),2)
        NewRow = {'Year': Year, 'Team': Team, 'Safeties': Safeties,'Corners': Corners,'InsideBackers': InsideBackers,'Edge': Edge, 'InteriorDLine': InteriorDLine,'WRs': WRs, 'TEs': TEs, 'Tackles': Tackles, 'InteriorOLine': InteriorOLine,'QB': QB,'RB': RB, 'Special': Special, 'Dead': Dead, 'IR': IR, 'Prac': Prac, 'Susp': Susp}
        GB12Buck = GB12Buck.append(NewRow, ignore_index = True)

In [5]:
## Change team and mascot to just mascot, which never changes
def JustMascot(x):
    Team = x['Team']
    Mascot = Team.split('-')[-1]
    return Mascot
    
    
    
GB12Buck['Team'] = GB12Buck.apply(JustMascot, axis = 1)

NameError: name 'GB12Buck' is not defined

In [38]:
## Merge team salary info with the success
GB12Buck = GB12Buck.merge(TeamsWins, how = "inner", on = ['Year','Team'])
GB12Buck.to_csv("12Buckets.csv", index = False)

In [33]:
GB12Buck

,Year,Team,Safeties,Corners,InsideBackers,Edge,InteriorDLine,WRs,TEs,InteriorOLine,...,QB,RB,Special,Dead,IR,Prac,Susp,Wins,Playoff Wins,SuperBowl Win
0,2005,cardinals,0.0,3.09,2.08,9.11,1.47,8.76,0.44,15.55,...,6.72,0.81,3.03,8.08,0.16,0.09,0.0,5,0,0
1,2006,cardinals,0.0,0.95,2.46,11.08,3.72,11.23,0.81,20.40,...,6.87,10.18,1.76,1.18,0.61,0.15,0.0,5,0,0
2,2007,cardinals,0.0,1.48,3.39,11.32,1.64,10.80,0.95,10.49,...,8.55,7.57,1.13,2.07,13.01,0.08,0.0,8,0,0
3,2008,cardinals,0.0,2.29,9.81,8.31,3.75,11.07,1.67,11.33,...,7.49,7.27,0.80,4.09,0.23,0.32,0.0,9,3,0
4,2009,cardinals,0.0,8.03,10.46,6.54,7.00,14.14,0.66,9.37,...,12.29,4.53,2.21,6.92,5.68,0.21,0.0,10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,2015,redskins,0.0,8.43,3.80,14.27,12.29,16.91,4.20,10.65,...,6.06,2.44,1.53,8.93,19.73,0.00,0.0,9,0,0
476,2016,redskins,0.0,10.29,2.04,9.03,4.87,14.27,5.11,9.01,...,13.73,1.51,1.61,0.37,7.29,0.00,0.0,8,0,0
477,2017,redskins,0.0,15.41,4.64,14.17,2.20,6.19,6.69,8.07,...,15.60,2.59,1.84,0.06,26.81,0.20,0.0,7,0,0
478,2018,redskins,0.0,11.73,3.38,11.99,4.71,6.27,9.13,6.16,...,12.49,3.79,2.53,0.40,0.14,30.37,0.0,7,0,0


In [7]:
Df = pd.read_csv("12Buckets.csv")

In [8]:
Df

,Year,Team,Safeties,Corners,InsideBackers,Edge,InteriorDLine,WRs,TEs,InteriorOLine,...,QB,RB,Special,Dead,IR,Prac,Susp,Wins,Playoff Wins,SuperBowl Win
0,2005,cardinals,8.02,3.09,2.08,9.11,1.47,8.76,0.44,15.55,...,6.72,0.81,3.03,8.08,0.16,0.09,0.0,5,0,0
1,2006,cardinals,9.22,0.95,2.46,11.08,3.72,11.23,0.81,20.40,...,6.87,10.18,1.76,1.18,0.61,0.15,0.0,5,0,0
2,2007,cardinals,8.73,1.48,3.39,11.32,1.64,10.80,0.95,10.49,...,8.55,7.57,1.13,2.07,13.01,0.08,0.0,8,0,0
3,2008,cardinals,8.13,2.29,9.81,8.31,3.75,11.07,1.67,11.33,...,7.49,7.27,0.80,4.09,0.23,0.32,0.0,9,3,0
4,2009,cardinals,10.10,8.03,10.46,6.54,7.00,14.14,0.66,9.37,...,12.29,4.53,2.21,6.92,5.68,0.21,0.0,10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,2015,redskins,5.14,8.43,3.80,14.27,12.29,16.91,4.20,10.65,...,6.06,2.44,1.53,8.93,19.73,0.00,0.0,9,0,0
476,2016,redskins,1.54,10.29,2.04,9.03,4.87,14.27,5.11,9.01,...,13.73,1.51,1.61,0.37,7.29,0.00,0.0,8,0,0
477,2017,redskins,2.81,15.41,4.64,14.17,2.20,6.19,6.69,8.07,...,15.60,2.59,1.84,0.06,26.81,0.20,0.0,7,0,0
478,2018,redskins,3.17,11.73,3.38,11.99,4.71,6.27,9.13,6.16,...,12.49,3.79,2.53,0.40,0.14,30.37,0.0,7,0,0


In [45]:
Df['CapRoom'] = 100 - (Df['Safeties'] + Df['Corners'] + Df['InsideBackers'] + Df['Edge'] + Df['InteriorDLine'] + Df['WRs'] + Df['TEs'] + Df['InteriorOLine'] + Df['Tackles'] + Df['QB'] + Df['RB'] + Df['Special'])

In [51]:
CapRoomDf = Df[['Year', 'Team', 'CapRoom']]

In [50]:
hist(CapRoomDF['CapRoom'])

In [43]:
CapRoomDf[(CapRoomDf['CapRoom'] < 0)&(CapRoomDf['Year'] != 2010)].sort_values(by = 'CapRoom')

,Year,Team,CapRoom
232,2012,chiefs,-19.38
442,2012,buccaneers,-14.15
412,2012,49ers,-14.15
142,2012,broncos,-10.21
367,2012,jets,-10.13
...,...,...,...
82,2012,bears,-1.25
73,2018,panthers,-1.18
303,2008,vikings,-1.02
202,2012,colts,-0.47


In [48]:
Df[(Df['Team'] == 'chiefs')&(Df['Year'] == 2012)]

,Year,Team,Safeties,Corners,InsideBackers,Edge,InteriorDLine,WRs,TEs,InteriorOLine,Tackles,QB,RB,Special,Dead,IR,Prac,Susp,Wins,Playoff Wins,SuperBowl Win,CapRoom
232,2012,chiefs,8.86,18.12,6.29,31.8,2.42,14.94,4.12,5.12,6.59,8.27,8.29,4.56,5.02,20.81,0.0,0.0,2,0,0,-19.38


In [53]:
CapRoomDf.to_csv('CapSpace.csv')